In [32]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
df = pd.read_csv('/content/drive/My Drive/Resources/kingcounty/seattle_sales_cleaned_data.csv')
# Review the DataFrame
df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,year
0,0,1/2/2018,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
1,1,1/9/2018,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
2,2,2/14/2018,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018
3,3,2/13/2018,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018
4,4,2/14/2018,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,24945,12/8/2022,895000,47.69,-122.33,SEATTLE,NR3,440000,327000,1926,3000,1100,7,7,3,1.50,3,2,0,0,370,0,0,0,0,0,0,0,0,0,0,0,0,0,2022
24934,24946,12/5/2022,1900000,47.60,-122.29,SEATTLE,NR3,790000,1432000,1924,5016,3000,10,8,3,2.00,4,2,1,0,240,0,0,0,0,0,0,0,2,2,0,0,1,0,2022
24935,24947,12/17/2022,1750000,47.63,-122.31,SEATTLE,NR3,853000,727000,1908,4800,2900,9,8,3,1.50,4,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2022
24936,24948,12/15/2022,1250000,47.68,-122.39,SEATTLE,NR3,497000,532000,1928,3825,1730,8,8,3,1.00,3,1,1,0,180,0,0,0,0,0,0,0,0,0,0,0,0,0,2022


In [34]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df['sale_month'] = df['sale_date'].dt.month
df['sale_day'] = df['sale_date'].dt.day
df.rename(columns={'year': 'sale_year'}, inplace=True)
df.head()

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
3,3,2018-02-13,1150000,47.72,-122.34,SEATTLE,SF 7200,440000,221000,1920,47916,1590,7,0,3,1.50,3,1,0,0,0,0,8,0,0,2,0,0,0,0,0,0,0,4,2018,2,13
4,4,2018-02-14,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14


In [5]:
#From the df, select all sales prices above 1,000,000 into one dataframe, and the rest into another one
#Re-run everything again the same way except use the new dfs

In [35]:
less_df = df[df["sale_price"] < 1000000]
less_df

,id,sale_date,sale_price,latitude,longitude,city,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,2018-01-02,665000,47.56,-122.32,SEATTLE,SF 5000,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,2018-01-09,499000,47.52,-122.32,SEATTLE,SF 5000,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,2018-02-14,675000,47.62,-122.30,SEATTLE,SF 5000,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
4,4,2018-02-14,705000,47.67,-122.32,SEATTLE,SF 5000,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14
5,5,2018-02-15,650000,47.67,-122.28,SEATTLE,SF 5000,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,24919,2022-09-30,999950,47.51,-122.26,SEATTLE,NR3,176000,686000,1907,5650,2980,8,8,3,1.50,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,30
24920,24932,2022-11-01,780000,47.56,-122.37,SEATTLE,NR3,266000,432000,1920,5360,1660,7,7,3,1.00,3,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2022,11,1
24926,24938,2022-11-11,675000,47.72,-122.33,SEATTLE,NR2,523000,310000,1953,7938,1310,7,0,3,1.00,3,1,0,0,0,260,0,0,0,0,0,0,0,0,0,0,0,0,2022,11,11
24932,24944,2022-12-06,845000,47.71,-122.32,SEATTLE,NR2,367000,474000,1945,7320,2330,7,6,3,1.00,3,2,1,0,370,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,6


In [36]:
less_df['zoning'].value_counts()
category_mapping = {
    'NR3': 1,
    'NR2': 2,
    'SF 5000': 3,
    'RSL (M)': 4,
    'SF 7200': 5,
    'NR1': 6,
    'SF 9600': 7
}

# Replace categorical values with integer values using the mapping
less_df['zoning'] = less_df['zoning'].map(category_mapping)
less_df['zoning'].value_counts()

<ipython-input-36-29e8cdb7a623>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['zoning'] = less_df['zoning'].map(category_mapping)


1    10230
2     2810
4     1103
3      740
5      162
6       79
7        6
Name: zoning, dtype: int64

In [37]:
less_df['sale_date'] = pd.to_datetime(df['sale_date'])
less_df['sale_date'] = less_df['sale_date'].astype('int64') // 10**9
less_df['sale_date']

<ipython-input-37-8681f4f75194>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['sale_date'] = pd.to_datetime(df['sale_date'])
<ipython-input-37-8681f4f75194>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_df['sale_date'] = less_df['sale_date'].astype('int64') // 10**9


0        1514851200
1        1515456000
2        1518566400
4        1518566400
5        1518652800
            ...    
24907    1664496000
24920    1667260800
24926    1668124800
24932    1670284800
24933    1670457600
Name: sale_date, Length: 15130, dtype: int64

In [38]:
less_df = less_df.drop(columns=['city'])
less_df

,id,sale_date,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month,sale_day
0,0,1514851200,665000,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,2
1,1,1515456000,499000,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1,9
2,2,1518566400,675000,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2,14
4,4,1518566400,705000,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2,14
5,5,1518652800,650000,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24907,24919,1664496000,999950,47.51,-122.26,1,176000,686000,1907,5650,2980,8,8,3,1.50,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,9,30
24920,24932,1667260800,780000,47.56,-122.37,1,266000,432000,1920,5360,1660,7,7,3,1.00,3,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2022,11,1
24926,24938,1668124800,675000,47.72,-122.33,2,523000,310000,1953,7938,1310,7,0,3,1.00,3,1,0,0,0,260,0,0,0,0,0,0,0,0,0,0,0,0,2022,11,11
24932,24944,1670284800,845000,47.71,-122.32,2,367000,474000,1945,7320,2330,7,6,3,1.00,3,2,1,0,370,0,0,0,0,1,0,0,0,0,0,0,0,0,2022,12,6


In [39]:
less_df = less_df.drop(columns=['sale_date'])

In [54]:
less_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15130 entries, 0 to 24933
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                15130 non-null  int64  
 1   sale_price        15130 non-null  int64  
 2   latitude          15130 non-null  float64
 3   longitude         15130 non-null  float64
 4   zoning            15130 non-null  int64  
 5   land_val          15130 non-null  int64  
 6   imp_val           15130 non-null  int64  
 7   year_built        15130 non-null  int64  
 8   sqft_lot          15130 non-null  int64  
 9   sqft              15130 non-null  int64  
 10  grade             15130 non-null  int64  
 11  fbsmt_grade       15130 non-null  int64  
 12  home_condition    15130 non-null  int64  
 13  stories           15130 non-null  float64
 14  beds              15130 non-null  int64  
 15  bath_full         15130 non-null  int64  
 16  bath_3qtr         15130 non-null  int64 

In [55]:
# Specify the column to exclude from the subset
column_to_exclude = 'sale_day'

# Create a subset of columns by excluding the specified column
subset_columns = [col for col in less_df.columns if col != column_to_exclude]

# Create a new DataFrame with the subset of columns
df_subset = less_df[subset_columns]

# Remove duplicate rows based on the subset of columns
df_no_duplicates = df_subset.drop_duplicates()
less_df = df_no_duplicates

In [56]:
summary_stats = less_df.describe()
print(summary_stats)

            id  sale_price  latitude  longitude   zoning   land_val  \
count 15130.00    15130.00  15130.00   15130.00 15130.00   15130.00   
mean  11822.51   722548.17     47.61    -122.34     1.57  390503.67   
std    6550.31   163015.38      0.08       0.04     1.01  171654.16   
min       0.00    65000.00     47.50    -122.42     1.00       0.00   
25%    6298.25   610000.00     47.54    -122.37     1.00  275000.00   
50%   11260.00   735000.00     47.61    -122.35     1.00  375000.00   
75%   17166.75   850000.00     47.69    -122.30     2.00  497000.00   
max   24945.00   999999.00     47.73    -122.24     7.00 1526000.00   

         imp_val  year_built  sqft_lot     sqft    grade  fbsmt_grade  \
count   15130.00    15130.00  15130.00 15130.00 15130.00     15130.00   
mean   373656.05     1942.55   5784.33  1566.11     6.90         3.42   
std    174671.36       25.09   2813.53   563.39     0.69         3.24   
min         0.00     1900.00    773.00   260.00     3.00         0.0

In [64]:
columns = ['sale_price', 'sqft_lot', 'sqft', 'land_val', 'imp_val']

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Calculate IQR for the specified columns
Q1 = less_df[columns].quantile(0.25)
Q3 = less_df[columns].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for the specified columns
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers for the specified columns
filtered_df = less_df[~((less_df[columns] < lower_bound) | (less_df[columns] > upper_bound)).any(axis=1)]
filtered_df = filtered_df.reset_index(drop=True)
filtered_df.head()

,id,sale_price,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,0,665000,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,1,499000,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,2,675000,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,4,705000,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,5,650000,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [65]:
y = filtered_df["sale_price"].ravel()
scaler = StandardScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).ravel()
X = filtered_df.copy()
X.drop("sale_price", axis=1, inplace=True)

In [66]:
y_scaled[:5]

array([-0.35221718, -1.39137293, -0.28961743, -0.1018182 , -0.44611679])

In [67]:
X.dropna()
X.head()

,id,latitude,longitude,zoning,land_val,imp_val,year_built,sqft_lot,sqft,grade,fbsmt_grade,home_condition,stories,beds,bath_full,bath_3qtr,bath_half,garb_sqft,gara_sqft,wfnt,golf,greenbelt,noise_traffic,view_rainier,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_otherwater,sale_year,sale_month
0,0,47.56,-122.32,3,41000,122000,1920,2525,1570,6,4,4,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
1,1,47.52,-122.32,3,29000,55000,1907,6000,1350,5,0,3,1.50,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,1
2,2,47.62,-122.30,3,97000,132000,1908,5286,1800,7,5,4,1.00,4,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2018,2
3,4,47.67,-122.32,3,81000,72000,1918,3825,780,6,0,3,1.00,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2018,2
4,5,47.67,-122.28,3,93000,100000,1948,6750,830,7,0,4,1.00,2,1,0,0,230,0,0,0,0,2,0,0,0,0,0,0,0,0,2018,2


In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

In [69]:
regression_model = LinearRegression()

# Fit the model using training data
regression_model.fit(X_train, y_train)

# Predict values using the trained model
y_pred = regression_model.predict(X_test)

In [70]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 0.4600962417652036
R-squared: 0.5477330633974598


In [71]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [72]:
y_pred2 = rf_model.predict(X_test)

In [73]:
pd.options.display.float_format = '{:.2f}'.format
y_pred2_original = scaler.inverse_transform(y_pred2.reshape(-1, 1)).ravel()

# Inverse transform the scaled actual prices to get original prices
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).ravel()

mse = mean_squared_error(y_test_original, y_pred2_original)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_original, y_pred2_original)
rmse_rounded = round(rmse) # Round and convert to integer

absolute_diff = np.abs(y_test_original - y_pred2_original)

# Create a DataFrame for comparison
comparison_df = pd.DataFrame({
    'Actual Price': y_test_original,
    'Predicted Price': y_pred2_original,
    'Absolute Difference': absolute_diff
})

# Print the comparison DataFrame
print(comparison_df)
print("Root Mean Squared Error:", rmse_rounded)
print("R-squared:", r2)

      Actual Price  Predicted Price  Absolute Difference
0        845000.00        831894.39             13105.61
1        590000.00        644477.79             54477.79
2        605000.00        884605.98            279605.98
3        660000.00        687057.50             27057.50
4        760000.00        721409.69             38590.31
...            ...              ...                  ...
2863     925000.00        898036.07             26963.93
2864     900000.00        845681.94             54318.06
2865     533000.00        572014.40             39014.40
2866     740200.00        828707.18             88507.18
2867     689000.00        662210.34             26789.66

[2868 rows x 3 columns]
Root Mean Squared Error: 92304
R-squared: 0.6718077971369982
